In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'd:\\Projects\\Cancer-classfiication-using-MLflow-and-DVC'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    valid_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmented: bool
    params_image_size: list

In [4]:
from src.ChestCancerClassfication.constants import *
from src.ChestCancerClassfication.utils.common import read_yaml, create_directories
import tensorflow as tf

In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, r"Data\train")
        valid_data = os.path.join(self.config.data_ingestion.unzip_dir, r"Data\valid")
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            valid_data = Path(valid_data),
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmented = params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE
        )

        return training_config

In [6]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [7]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        if self.config.params_is_augmented:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                **datagenerator_kwargs
            )

        self.train_generator = train_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            shuffle=True,
            **dataflow_kwargs
        ) 

        # No augmentation for validation set
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.valid_data,
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
    )

In [8]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-03-15 18:19:11,622: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2024-03-15 18:19:11,631: INFO: common: yaml file: params.yaml loaded successfully.]
[2024-03-15 18:19:11,632: INFO: common: created directory at: artifacts]
[2024-03-15 18:19:11,632: INFO: common: created directory at: artifacts\training]
Found 613 images belonging to 4 classes.
Found 72 images belonging to 4 classes.
Epoch 1/10
38/38 [==============================] - 104s 3s/step - loss: 39.5778 - accuracy: 0.2680 - val_loss: 50.9842 - val_accuracy: 0.1094
Epoch 2/10
38/38 [==============================] - 106s 3s/step - loss: 30.8458 - accuracy: 0.3149 - val_loss: 26.6316 - val_accuracy: 0.5625
Epoch 3/10
38/38 [==============================] - 104s 3s/step - loss: 24.8110 - accuracy: 0.3802 - val_loss: 11.4636 - val_accuracy: 0.5469
Epoch 4/10
38/38 [==============================] - 103s 3s/step - loss: 22.9343 - accuracy: 0.3936 - val_loss: 16.6754 - val_accuracy: 0.4688
Epoch 5/10
38/3

: 